# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz,circular_ansatz_mirrored
from clapton.evaluation import get_energy
import numpy as np
from numbers import Number


In [2]:
def ising_model(N, Jx, h, Jy=0., periodic=False):
    """
    Constructs qubit Hamiltonian for linear Ising model.
    H = sum_{i=0...N-2} (Jx_i X_i X_{i+1} + Jy_i Y_i Y_{i+1}) + sum_{i=0...N-1}  h_i Z_i

    N (Int): # sites/qubits.
    Jx (Float, Iterable[Float]): XX strength, either constant value or list (values for each pair of neighboring sites).
    h (Float, Iterable[Float]): Z self-energy, either constant value or list (values for each site).
    Jy (Float, Iterable[Float]): YY strength, either constant value or list (values for each pair of neighboring sites).
    periodic: If periodic boundary conditions. If True, include term X_0 X_{N-1} and Y_0 Y_{N-1}.

    Returns:
    (Iterable[Float], Iterable[String], String) (Pauli coefficients, Pauli strings, "0"*N)
    """
    if isinstance(Jx, Number):
        if periodic:
            Jx = [Jx] * N
        else:
            Jx = [Jx] * (N-1)
    if isinstance(Jy, Number):
        if periodic:
            Jy = [Jy] * N
        else:
            Jy = [Jy] * (N-1)
    if isinstance(h, Number):
        h = [h] * N        
    if N > 1:
        assert len(Jx) == N if periodic else len(Jx) == N-1, "Jx has wrong length"
        assert len(Jy) == N if periodic else len(Jy) == N-1, "Jy has wrong length"
        assert len(h) == N, "h has wrong length"
    coeffs = []
    paulis = []
    # add XX terms
    for j in range(N-1):
        if np.abs(Jx[j]) > 1e-12:
            coeffs.append(Jx[j])
            paulis.append("I"*j+"XX"+"I"*(N-j-2))
    if N > 2 and periodic and np.abs(Jx[N-1]) > 1e-12:
        coeffs.append(Jx[N-1])
        paulis.append("X"+"I"*(N-2)+"X")
    # add YY terms
    for j in range(N-1):
        if np.abs(Jy[j]) > 1e-12:
            coeffs.append(Jy[j])
            paulis.append("I"*j+"YY"+"I"*(N-j-2))
    if N > 2 and periodic and np.abs(Jy[N-1]) > 1e-12:
        coeffs.append(Jy[N-1])
        paulis.append("Y"+"I"*(N-2)+"Y")
    # add Z terms
    for j in range(N):
        if np.abs(h[j]) > 1e-12:
            coeffs.append(h[j])
            paulis.append("I"*j+"Z"+"I"*(N-j-1))
    return coeffs, paulis, "1"*N

In [3]:
# set seed for reproducibility
np.random.seed(42)

# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))
paulis, coeffs

# 10 Qubit Hamiltonian
# coeffs,paulis,_ = ising_model(N=10,Jx=0.2,Jy=0.3,h=0.4)



(['XXI', 'IXX', 'YYI', 'IYY', 'ZZI', 'IZZ'],
 array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
        0.15599452]))

In [4]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_algorithms import NumPyMinimumEigensolver

weights  =  coeffs
pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,weights)]
hamiltonian = SparsePauliOp.from_list([ op for op in pauli_op ])

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

Reference value: -2.06783


In [5]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=10, fix_2q=True)

In [6]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [7]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which describe multiples of pi/2

In [8]:
# the initial parameters are all 0
vqe_pcirc.read()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---
         | |       | |       | |       | |       | |       | |       | |       | |       | |       | |
q1: -I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-
         |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |
q2: -I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-

In [10]:
# we can assign a different set of parameters
# vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [11]:
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---X-@-I-I---
         | |       | |       | |       | |       | |       | |       | |       | |       | |       | |
q1: -I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-|-X-@-I-I-
         |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |     |   |
q2: -I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-@---X-I-I-

In [12]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs




/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


started GA at id 3 with 1 procs


/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


started GA at id None with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

[0, array([1.19731697, 0.59865848, 0.59865848, 0.        ]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 3, 0,
       2, 3, 2, 1, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2, 1, 2, 0, 1, 1, 1, 1, 3,
       0, 0, 2, 3, 0, 2, 2, 0, 2, 1, 2, 3, 0, 3, 2, 1, 2, 1, 1, 1, 0, 2],
      dtype=object)]
[0, array([1.19731697, 0.59865848, 0.59865848, 0.        ]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 3, 0,
       2, 3, 2, 1, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2, 1, 2, 0, 1, 1, 1, 1, 3,
       0, 0, 2, 3, 0, 2, 2, 0, 2, 1, 2, 3, 0, 3, 2, 1, 2, 1, 1, 1, 0, 2],
      dtype=object)]
[0, array([1.19731697, 0.59865848, 0.59865848, 0.        ]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 3, 0,
       2, 3, 2, 1, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2, 1, 2, 0, 1, 1, 1, 1, 3,
       0, 0, 2, 3, 0, 2, 2, 0, 2, 1, 2, 3, 0, 3, 2, 1, 2, 1, 1, 1, 0, 2],
      dtype=object)]
[0, array([1.19731697, 0.59865848, 0.59865848, 0.        ]), array([3, 3, 0, 2, 3, 3, 2, 3, 2,

In [13]:
# the best parameters are
ks_best

[0,
 2,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 3,
 2,
 3,
 1,
 0,
 1,
 1,
 2,
 3,
 2,
 3,
 2,
 1,
 3,
 1,
 1,
 3,
 3,
 3,
 2,
 1,
 3,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 3,
 0,
 3,
 3,
 1,
 1,
 3,
 1,
 2,
 0,
 3,
 3,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 3,
 2,
 1,
 3,
 1,
 2,
 2]

In [14]:
# with energy
energy_best

0.5986584841970366

In [15]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -I------I-X-@-I-I-------X-@-Y-I------------X-@-Y-S_DAG--------X-@-SQRT_Y_DAG-S_DAG--------X-@-Y-I------------X-@-SQRT_Y-I-------X-@-SQRT_Y_DAG-S----------------X-@-Y-S_DAG--------X-@-Y-I-------X-@-Y-S------------
              | |           | |                | |                | |                         | |                | |                | |                             | |                | |           | |
q1: -Y------I-|-X-@-Y-I-----|-X-@-SQRT_Y_DAG-S-|-X-@-SQRT_Y_DAG-Z-|-X-@----------SQRT_Y-S_DAG-|-X-@-SQRT_Y-----S-|-X-@------Y-S_DAG-|-X-@----------SQRT_Y_DAG-S_DAG-|-X-@-I----------S-|-X-@-I-S-----|-X-@-SQRT_Y-----Z-
              |   |         |   |              |   |              |   |                       |   |              |   |              |   |                           |   |              |   |         |   |
q2: -SQRT_Y-S-@---X-I-S_DAG-@---X-SQRT_Y-----S-@---X-Y----------S-@---X----------SQRT_Y-S_DAG-@---X-SQRT_Y_DAG-Z-@---X------Y-I-----@---X----------SQRT_Y-----S-----@---X-SQRT_Y_DAG-I-@---X-Y-S_DAG-@---X-SQRT_Y_DAG-Z-

# noisy CAFQA (nCAFQA)

In [16]:
from clapton.depolarization import GateGeneralDepolarizationModel

In [17]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.05) #NOTE: This is the Noise Model, not representative of device noise

In [18]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=10, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [19]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |                                                                                                                                                       | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-I------------------DEPOLARIZE1(0.005)--------------------I--------------

In [20]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs

started GA at id 3 with 1 procs
started GA at id None with 1 procs


[0, array([0.66246497, 0.06380649, 0.59865848, 0.        ]), array([1, 2, 1, 0, 2, 1, 0, 2, 1, 0, 0, 3, 3, 2, 2, 0, 1, 0, 3, 2, 2, 1,
       2, 2, 0, 3, 1, 0, 0, 3, 2, 2, 2, 3, 0, 2, 2, 1, 2, 2, 2, 3, 1, 0,
       3, 2, 3, 0, 0, 2, 0, 3, 1, 0, 1, 0, 2, 1, 2, 1, 0, 1, 3, 2, 2, 0],
      dtype=object)][0, array([0.65839259, 0.0597341 , 0.59865848, 0.        ]), array([0, 3, 2, 2, 2, 0, 1, 3, 2, 1, 1, 0, 3, 2, 0, 1, 1, 1, 0, 2, 3, 0,
       2, 1, 3, 1, 3, 0, 3, 1, 2, 2, 2, 0, 3, 0, 2, 3, 3, 2, 1, 1, 2, 3,
       3, 2, 1, 1, 0, 3, 1, 1, 1, 0, 2, 3, 0, 0, 0, 2, 1, 1, 2, 3, 3, 2],
      dtype=object)]

[0, array([0.66011539, 0.0614569 , 0.59865848, 0.        ]), array([3, 3, 1, 0, 2, 1, 0, 3, 3, 0, 0, 3, 2, 1, 3, 0, 1, 3, 3, 3, 3, 3,
       3, 0, 2, 1, 1, 2, 0, 3, 1, 3, 1, 1, 2, 0, 3, 2, 3, 2, 1, 2, 0, 3,
       1, 2, 1, 2, 0, 2, 3, 1, 0, 1

In [21]:
# the best parameters are
ks_best

[0,
 0,
 1,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 1,
 3,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 3,
 0,
 1,
 3,
 1,
 3,
 0,
 3,
 1,
 0,
 3,
 2,
 2,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 1,
 3,
 0,
 3,
 2,
 0,
 2,
 2,
 1,
 3,
 0,
 0,
 3,
 1,
 3,
 2,
 2,
 3,
 0]

In [22]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(0.0932051265125863, 0.5986584841970366)

In [23]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.5054533576844503

In [24]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -I------S_DAG-X-@-SQRT_Y_DAG-S_DAG------------X-@-Y-S--------X-@-I-I---X-@-SQRT_Y_DAG-S_DAG--------X-@-I-I----------------X-@-Y-S_DAG--------X-@-SQRT_Y_DAG-Z---X-@-SQRT_Y_DAG-Z------------X-@-Y-I----------------X-@-SQRT_Y-Z----------------
                  | |                             | |            | |       | |                         | |                    | |                | |                | |                         | |                    | |
q1: -I------S_DAG-|-X-@----------SQRT_Y-----S-----|-X-@-I------I-|-X-@-Y-Z-|-X-@----------I------S-----|-X-@-SQRT_Y_DAG-S_DAG-|-X-@-SQRT_Y_DAG-I-|-X-@----------I-I-|-X-@----------I----------I-|-X-@-SQRT_Y-----I-----|-X-@------SQRT_Y_DAG-S_DAG-
                  |   |                           |   |          |   |     |   |                       |   |                  |   |              |   |              |   |                       |   |                  |   |
q2: -SQRT_Y-I-----@---X----------SQRT_Y_DAG-S_DAG-@---X-SQRT_Y-S-@---X-I-I-@---X----------SQRT_Y-S_DAG-@---X-SQRT_Y-----Z-----@---X-I----------I-@---X----------I-S-@---X----------SQRT_Y_DAG-Z-@---X-SQRT_Y_DAG-S_DAG-@---X------Y----------I-----

## Implementing PT after CAFQA 

In [25]:
pauli_twirl_list = [vqe_pcirc.add_pauli_twirl() for _ in range(1000)]
for circuit in pauli_twirl_list:
    circuit.assign(ks_best)
pauli_twirl_list = [circuit.add_depolarization_model(nm) for circuit in pauli_twirl_list]
vqe_pcirc.add_pauli_twirl_list(pauli_twirl_list) #NOTE: Made major change here by adding list after adding noise

In [26]:
pt_energy = get_energy(vqe_pcirc,paulis,coeffs)
pt_energy

-0.05729200455637648

## Comparing Initial Energy States

In [27]:
pt_energy<energy_noisy

True

In [28]:
pt_energy,energy_noisy

(-0.05729200455637648, 0.0932051265125863)

## Let's do one VQE iteration now

In [29]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit import ParameterVector, QuantumCircuit
import qiskit_algorithms
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2,FakeTorontoV2


def qiskit_circular_ansatz(N, reps=1, fix_2q=False): 
    qc = QuantumCircuit(N)

    # define your parameters
    p = ParameterVector('p', len(ks_best)) 

    for r in range(reps): 
        for i in range(N):
            qc.ry(p[2*N*r+i], i)  
        for i in range(N):
            qc.rz(p[2*N*r+(i+N)], i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(p[2*N*reps+i], i)
    for i in range(N):
        qc.rz(p[2*N*reps + (i+N)], i)
    return qc

num_qubits = len(paulis[0])
qc = qiskit_circular_ansatz(num_qubits,reps=10)

# bind the values
# bound_circuit = qc.assign_parameters(ks_best)



In [30]:
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer.noise import NoiseModel

device = GenericBackendV2(num_qubits=num_qubits,seed=0)

noise_model = NoiseModel.from_backend(device)

print(noise_model)


NoiseModel:
  Basis gates: ['cx', 'delay', 'id', 'measure', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['cx', 'x', 'sx', 'id', 'measure']
  Qubits with noise: [0, 1, 2]
  Specific qubit errors: [('cx', (0, 1)), ('cx', (1, 0)), ('cx', (0, 2)), ('cx', (2, 0)), ('cx', (1, 2)), ('cx', (2, 1)), ('id', (0,)), ('id', (1,)), ('id', (2,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('x', (0,)), ('x', (1,)), ('x', (2,)), ('measure', (0,)), ('measure', (1,)), ('measure', (2,))]


In [ ]:
# #EstimatorV1 without Pauli Twirling API 

# # from qiskit.primitives import Estimator
# from qiskit_aer.primitives import Estimator
# # Create the ansatz circuit
# ansatz = qc 

# #Set seed for optimizer 
# qiskit_algorithms.utils.algorithm_globals.random_seed = 0

# # Set up the optimizer
# optimizer = SPSA(maxiter=1)

# # Create the estimator
# estimator = Estimator(backend_options={"noise_model":noise_model},
#                       run_options={"seed": 0},
#                       transpile_options={"seed_transpiler": 0},
#                       )

# # Initialize and run VQE
# vqe = VQE(estimator, ansatz, optimizer,initial_point=ks_best)
# result = vqe.compute_minimum_eigenvalue(hamiltonian)

# print(f"Minimum eigenvalue: {result.eigenvalue.real:.5f}")
# print(f"Optimal parameters: {result.optimal_parameters.values()}")

In [33]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.options import EstimatorOptions
 
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)


backend = FakeMumbaiV2()

options = EstimatorOptions(
    default_shots=1024,resilience_level=0)
    
ansatz = qc 

#Set seed for optimizer 
qiskit_algorithms.utils.algorithm_globals.random_seed = 0

# Set up the optimizer
optimizer = SPSA(maxiter=1)
 
estimator = Estimator(mode=backend, options=options)

# Initialize and run VQE
vqe = VQE(estimator, ansatz, optimizer,initial_point=ks_best)
result = vqe.compute_minimum_eigenvalue(hamiltonian)

print(f"Minimum eigenvalue: {result.eigenvalue.real:.5f}")
print(f"Optimal parameters: {result.optimal_parameters.values()}")

AlgorithmError: 'The primitive job to evaluate the energy failed!'

In [31]:
ref_value

-2.067831908873277